In [ ]:
import ginsim
import biolqm
from colomoto_jupyter import tabulate
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

try:
    import cobra
except ImportError:
    import sys
    !{sys.executable} -m pip install cobra
    import cobra

cmp = sns.diverging_palette(0, 255, as_cmap=True)

# 1. Loading the two models of interest

- The CaSQ-generated CAF-specific boolean model;
- MitoCore constraint-based model of human metabolism.

## 1. 1. Load the generic CAF-model

In [ ]:
CAF_model = biolqm.load("CAF-model.sbml")

The sanitize function is used to generate human-friendly node IDs and rescale the layout to improve the model's readability.


*Note: this cell may take a few minutes to run.*

In [ ]:
CAF_model = biolqm.sanitize(CAF_model)

layout = CAF_model.getLayout()
layout.scale(0.4)

lrg = biolqm.to_ginsim(CAF_model)

ginsim.show(lrg)

## 1.2. Load MitoCore 

In [ ]:
import logging
cobra.io.sbml.LOGGER.setLevel(logging.ERROR)

MitoCore = cobra.io.read_sbml_model('mitocore_v1.01.xml')

# 2. Value propagation

We show that the **input conditions** (here, based on literature) strongly control the whole network.

In [ ]:
inits = pd.read_csv("Breast_CAF_initial_conditions.tsv", sep="\t", index_col=0)
dic_inits = inits.to_dict()

Definition of some helper functions and color mapping rules to perform value propagation and visualize the result.

In [ ]:
# Transforms a dictionary into a dash-like pattern used for space restrictions.
# If a model has 4 components A, B, C, D in this order,
#  {A:0, D:1} => "0--1"

def dash_pattern(model, dict_vals):
    specific_comps = dict_vals.keys()
    str_pattern = ""
    for comp in model.getComponents():
        if comp.toString() in specific_comps:
            str_pattern += str(dict_vals.get(comp.toString()))
        else :
            str_pattern += "-"
    return(str_pattern)

def restrict_model(model, **dict_vals):
    pattern = dash_pattern(model, dict_vals)
    return biolqm.restrict(model, pattern)

def fill_fixed(data, names, functions, mddman):
    all_values = [f for f in functions]
    for comp, func in zip(names, functions):
        if mddman.isleaf(func): data[comp] = func
        else: data[comp] = -1
    

def get_fixed_pattern(all_names, model, as_dict=False):
    # Build a container for the results
    pattern = {key: 100 for key in all_names}
    
    # Model manager and core components
    mddman = model.getMDDManager()
    core_components = [node.getNodeID() for node in model.getComponents()]
    extra_components = [node.getNodeID() for node in model.getExtraComponents()]
    
    # 1/ Non-extra values: if the model was not reduced, core components may also contain fixed values
    fill_fixed(pattern, core_components, model.getLogicalFunctions(), mddman)
    
    # 2/ Extra values : only available after reduction/percolation
    # Functions of each component
    fill_fixed(pattern, extra_components, model.getExtraLogicalFunctions(), mddman)

    if as_dict: return pattern
    return pd.Series(pattern, dtype=np.byte)

def get_fixed(gs_model, restricted_model, as_dict=False):
    name_components = [ n.getId() for n in gs_model.getNodeOrder() ]
    return get_fixed_pattern(name_components, restricted_model, as_dict)

def show_fixed(gs_model, styler, fixed_pattern, save=None):
    styler.setState(fixed_pattern.values.tobytes())
    return ginsim.show(gs_model, style=styler, save=save)


# Define color mapping rules
styler_fixed = ginsim.lrg_style(lrg)
styler_fixed.mapState2Color(0, 255, 255, 255)
styler_fixed.mapState2Color(1, 100, 100, 255)
styler_fixed.mapState2Color(-1, 255, 100, 100)

In [ ]:
data = []

for init_name, values in dic_inits.items():
    lqm_model_restricted = restrict_model(CAF_model, **values)
    data.append( get_fixed(lrg, lqm_model_restricted) )
    
df = pd.concat(data, axis=1, keys=[name for name in dic_inits])

In [ ]:
df.to_csv("Breast_CAF_model_value_propagation.csv")

The result of value propagation can be visualized in the following heatmap where each line represents a component of the system and the column represent the input condition. 
    
- A **white cell** denotes that the corresponding component is **fixed at 0** by value propagation in this input condition;
- A **blue cell** denotes that the corresponding component is **fixed at 1** by value propagation in this input condition;
- A **red cells** denote components **which are not fixed** by value propagation in this input condition.

In [ ]:
plt.figure(figsize=(8,80))

sns.heatmap(df, center=0, cmap=cmp, cbar=False)

Propagated inputs for a specific input condition can also be mapped on the regulatory graph using the same color code.

In [ ]:
fixed = data[0]

show_fixed(lrg, styler_fixed, fixed, save="Breast_CAF_model_value_propagation_visualization.svg")

# 3. Identification of breast CAF-model's trap-spaces 

## 3.1. Using the output of value propagation as a new set of initial conditions

The biolqm.perturbation function enables the construction of a variant of the model, where the logical function of one or several components has been modified. A textual parameter describes the modification:

    component%0 defines a knockout of a component
    component%1 defines an ectopic expression
    
To perturbate the model, the output dataframe of value propagation is transformed as a list of perturbations in the form **'component1%value component2%value ... componentN%value'**.

In [ ]:
df = df[df.C1 >= 0]
df["modifs"]= df.index.map(str) + "%" + df["C1"].map(str) 
modifications = pd.DataFrame(df["modifs"]).copy()
modifications = modifications.reset_index(drop=True)
pert = modifications["modifs"].tolist()
perturbations = " ".join(pert)

In [ ]:
CAF_model_perturbated = biolqm.perturbation(CAF_model, perturbations)

## 3.2. Identification of CAF-model's regulatory trap-spaces

A trap-space, also called stable motif or called symbolic steady state, is a partially assigned state such that all possible successors of all states which belong to the motif also belong to the motif. Like stable states, these stable motifs can be identified efficiently using constraint-solving methods.

In [ ]:
trapspaces = biolqm.trapspace(CAF_model_perturbated)

In [ ]:
trapspaces_df = pd.DataFrame(trapspaces)
trapspaces_df

In [ ]:
trapspaces_df.to_csv("complete_breast_CAF_model_trapspaces.csv")

# 4. Projection of regulatory trap-spaces on metabolic compounds

## 4.1. Extraction of metabolic components 

To extract common enzymes and metabolites between MitoCore and the CAF-model we generate:

- The list of enzymes in MitoCore;
- The list of metabolites in MitoCore;
- The list of compounds (both enzymes and metabolites) in the breast CAF-model.

These list are used to compare both models and automatically extract common components (with their class: "metabolic enzyme" or "metabolite"). We exclude common metabolic intermediates.

In [ ]:
MitoCore_Enzymes = [r.id for r in MitoCore.reactions]

MitoCore_Metabolites = [m.id for m in MitoCore.metabolites]

CAF_model_components = [n.getName().replace('_simple_molecule', '').replace('_active', '').replace('M_', '').replace('_mitochondria', '').replace('_Cytosol', '') for n in CAF_model.getComponents()]

### 4.1.1. Extracting common metabolic enzymes

In [ ]:
common_enzymes = list((set(CAF_model_components).intersection(MitoCore_Enzymes)))

### 4.1.2. Extracting common metabolites

We limit the metabolite matching by excluding a list of predefined compounds which are considered by MitoCore as metabolites but are common metabolic intermediates.

In [ ]:
intermediates = ["atp_c", "adp_c", "adn_c", "adp_m", "amp_c", "amp_m", "atp_m", "cdp_m", "cmp_c", "co_c", "co_e",
                 "co2_c", "co2_e", "co2_m", "coa_c", "coa_m", "ctp_c", "fe2_c", "fe2_e", "fe2_m", "ficytC_c",
                 "ficytC_e", "ficytC_m", "gdp_c", "gdp_m", "gtp_c", "gtp_m", "h_c", "h_e", "h_m", "h2o_c", "h2o_m",
                 "h2o2_c"," h2o2_m", "hco3_c", "hco3_e", "hco3_m", "nad_c", "nad_e", "nad_m", "nadh_c", "nadh_e",
                 "nadh_m", "nadp_c", "nadp_m", "nadph_c", "nadph_m", "no_c", "no_e", "o2_c", "o2_e", "o2_m", "o2s_m",
                 "pheme_c", "pheme_m", "pi_c", "pi_e", "pi_m", "q10_m", "q10h2_m"]

In [ ]:
common_metabolites = list(set(CAF_model_components).intersection(MitoCore_Metabolites)-set(intermediates))

## 4.2. Projecting metabolic components' regulatory trap-spaces

### 4.2.1. Projecting metabolic enzymes' regulatory trap-spaces

In [ ]:
trapspaces_df.columns = trapspaces_df.columns.str.replace("_Cytoplasm","").str.replace("_simple_molecule","").str.replace("_mitochondria","").str.replace("M_","")

coltofiler = np.intersect1d(trapspaces_df.columns.values,common_enzymes)
trapspaces_metabolic_enzymes = trapspaces_df[coltofiler]

trapspaces_metabolic_enzymes
trapspaces_metabolic_enzymes.to_csv("Breast_CAF_model_trapspaces_metabolic_enzymes.csv")

In [ ]:
trapspaces_metabolic_enzymes

### 4.2.2. Projecting metabolites' regulatory trap-spaces

In [ ]:
coltofiler = np.intersect1d(trapspaces_df.columns.values,common_metabolites)
trapspaces_metabolites = trapspaces_df[coltofiler]

trapspaces_metabolites
trapspaces_metabolites.to_csv("Breast_CAF_model_trapspaces_metabolites.csv")

In [ ]:
trapspaces_metabolites

# 5. Flux Balance Analysis (FBA)

Definition of an **ATP_total** objective function representing maximum cellular ATP production through glycolysis and oxidative phosphorylation:

In [ ]:
MitoCore.objective = ["PYK", "PGK", "CV_MitoCore"]

## 5.1. FBA n°1: Control

In [ ]:
ATP_total_CTL = MitoCore.optimize().objective_value

In [ ]:
MitoCore.summary()

In [ ]:
solution = MitoCore.optimize()

ATP_glycolysis_CTL = (solution.fluxes['PYK'] + solution.fluxes['PGK'])/(solution.objective_value)

print("The proportion of global ATP production through glycolysis in control conditions is", round(ATP_glycolysis_CTL, 4), ".")

In [ ]:
pd.DataFrame(MitoCore.optimize().fluxes).to_csv('FBA_CTL_obj_ATP.csv')

## 5.2. FBA n°2: Breast CAF-specific

### 5.2.1. Constraining metabolic fluxes with regulatory trap-spaces 

#### 5.2.1.1. Constraining metabolic fluxes with maximum metabolic enzymes' regulatory trap-spaces equal to 0

In [ ]:
trapspaces_metabolic_enzymes_zero = trapspaces_metabolic_enzymes.loc[:,(trapspaces_metabolic_enzymes.max(axis=0)) == 0]
enzymes_to_zero = trapspaces_metabolic_enzymes_zero.columns.values.tolist()

enzymes_to_zero

Setting the constraints:

In [ ]:
for i in enzymes_to_zero:
    MitoCore.reactions.get_by_id(i).lower_bound = 0
    MitoCore.reactions.get_by_id(i).upper_bound = 0

#### 5.2.1.2. Constraining metabolic fluxes with maximal metabolites' regulatory trap-spaces eual to 0

In [ ]:
trapspaces_metabolites_zero = trapspaces_metabolites.loc[:,(trapspaces_metabolites.max(axis=0)) == 0]
metabolites_to_zero = trapspaces_metabolites_zero.columns.values.tolist()

metabolites_to_zero

In [ ]:
producing_reactions_metabolite_to_zero  = []

for i in metabolites_to_zero:
    producing_reactions_metabolite_to_zero1 = MitoCore.metabolites.get_by_id(i).summary().producing_flux.index.values.tolist()
    producing_reactions_metabolite_to_zero  = producing_reactions_metabolite_to_zero + producing_reactions_metabolite_to_zero1

In [ ]:
producing_reactions_metabolite_to_zero

Setting the constraints:

In [ ]:
for i in producing_reactions_metabolite_to_zero:
    MitoCore.reactions.get_by_id(i).lower_bound = 0
    MitoCore.reactions.get_by_id(i).upper_bound = 0

## 5.2. FBA

In [ ]:
ATP_total_CAF = MitoCore.optimize().objective_value

In [ ]:
MitoCore.summary()

In [ ]:
solution = MitoCore.optimize()

ATP_glycolysis_CAF = (solution.fluxes['PYK'] + solution.fluxes['PGK'])/(solution.objective_value)

print("The proportion of global ATP production through glycolysis in CAF-specific conditions is", round(ATP_glycolysis_CAF, 4), ".")

In [ ]:
pd.DataFrame(MitoCore.optimize().fluxes).to_csv('FBA_CAF_obj_ATP.csv')